# Email Data Pipeline

This notebook extracts data from qardio google sheets, cleans the data, and loads it into the Qardio SQL database

In [21]:
from google_apis import gsheets_functions as gs
import pandas as pd
import sqlalchemy

database = 'qardio'
host = '127.0.0.1'
user = 'root'
password = 'Party100'
url = f'mysql+mysqlconnector://{user}:{password}@{host}/{database}'
engine = sqlalchemy.create_engine(url, echo=True)
conn = engine.connect()

pd.set_option('display.max_columns', 500)

2023-06-22 13:42:23,016 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-06-22 13:42:23,017 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-06-22 13:42:23,023 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-06-22 13:42:23,026 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-06-22 13:42:23,032 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-06-22 13:42:23,033 INFO sqlalchemy.engine.Engine [raw sql] {}


In [22]:
qardio_data = gs.gspread_read(wb='1dKiIp2ETyzfKHzR9fMeHrgf2N_UKcSa5uPWathJzgIw',
                              ws='22_+_23')

Clean column headers

In [23]:
qardio_data.columns = qardio_data.columns.str.strip()
qardio_data.columns = qardio_data.columns.str.lower()
qardio_data.columns = qardio_data.columns.str.replace(' ', '_')
qardio_data.columns = qardio_data.columns.str.replace('.', '')
qardio_data = qardio_data.drop(columns=['year', 'day', 'time_pst'])

Solve dtypes

In [24]:
qardio_data.date_sent = pd.to_datetime(qardio_data.date_sent)
for col in qardio_data.loc[:,'recipients':].select_dtypes('object').columns:
    qardio_data[col] = qardio_data[col].str.replace('$', '')
    qardio_data[col] = qardio_data[col].str.replace('%', '')
    qardio_data[col] = qardio_data[col].str.replace(',', '')
    qardio_data[col] = qardio_data[col].str.replace('#DIV/0!', '')
    qardio_data[col] = pd.to_numeric(qardio_data[col])

In [25]:
qardio_data.dtypes

campaign_name                     object
date_sent                 datetime64[ns]
subject_line                      object
subscribers                       object
country                           object
recipients                         int64
opens                              int64
open_rate                        float64
clicks                           float64
click_rate                       float64
true_click_rate                  float64
click_per_unique_opens           float64
unsubs                             int64
%_unsubs                         float64
%_unsubs_openers                 float64
bounced                            int64
sessions                         float64
sess_per_unique_opens            float64
ecr                              float64
total_trans                      float64
revenue                          float64
aov                              float64
qardioarm                        float64
qardiobase_2                     float64
qardiobase_x    

## Save data to SQL database

In [26]:
qardio_data.to_sql(name='email_data',
          con=conn,
          if_exists='replace',
          index=False)

2023-06-22 13:42:25,389 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-22 13:42:25,409 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2023-06-22 13:42:25,417 INFO sqlalchemy.engine.Engine [generated in 0.01139s] {'table_schema': 'qardio', 'table_name': 'email_data'}
2023-06-22 13:42:25,431 INFO sqlalchemy.engine.Engine 
CREATE TABLE email_data (
	`index` BIGINT, 
	campaign_name TEXT, 
	date_sent DATETIME, 
	subject_line TEXT, 
	subscribers TEXT, 
	country TEXT, 
	recipients BIGINT, 
	opens BIGINT, 
	open_rate FLOAT(53), 
	clicks FLOAT(53), 
	click_rate FLOAT(53), 
	true_click_rate FLOAT(53), 
	click_per_unique_opens FLOAT(53), 
	unsubs BIGINT, 
	`%_unsubs` FLOAT(53), 
	`%_unsubs_openers` FLOAT(53), 
	bounced BIGINT, 
	sessions FLOAT(53), 
	sess_per_unique_opens FLOAT(53), 
	ecr FLOAT(53), 
	total_trans FLOAT(53), 
	revenue FLOAT(53), 
	aov FLOAT(53), 
	qardioarm FLOAT(53), 
	qard

2023-06-22 13:42:25,456 INFO sqlalchemy.engine.Engine CREATE INDEX ix_email_data_index ON email_data (`index`)
2023-06-22 13:42:25,460 INFO sqlalchemy.engine.Engine [no key 0.00417s] {}
2023-06-22 13:42:25,672 INFO sqlalchemy.engine.Engine INSERT INTO email_data (`index`, campaign_name, date_sent, subject_line, subscribers, country, recipients, opens, open_rate, clicks, click_rate, true_click_rate, click_per_unique_opens, unsubs, `%_unsubs`, `%_unsubs_openers`, bounced, sessions, sess_per_unique_opens, ecr, total_trans, revenue, aov, qardioarm, qardiobase_2, qardiobase_x, qardiotemp, qardiospo2, qa_case, `qa+qbx_bundle`, `qa+qb_bundle`, `qa+spo2`, `qtemp+spo2`, ultimate_bundle, qardio_core, total_quantity) VALUES (%(index)s, %(campaign_name)s, %(date_sent)s, %(subject_line)s, %(subscribers)s, %(country)s, %(recipients)s, %(opens)s, %(open_rate)s, %(clicks)s, %(click_rate)s, %(true_click_rate)s, %(click_per_unique_opens)s, %(unsubs)s, %(P_unsubs)s, %(P_unsubs_openers)s, %(bounced)s, %(s

1548